# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-267c3eb429-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Authors comparison explorative search") 

Consider the following exploratory scenario:


>  Investigate the the production of Paul Auster and Ian McEwan, check how many books they have written for each litarature genre, gather information about their production and about their works which are not books



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q47461344`| writtenwork   | node |
| `wd:Q214642`  | Paul Auster   | node |
| `wd:Q190379`  | Ian McEwan    | node |




Also consider

```
wd:Q214642 ?rel ?obj  . 
```
is the BGP to retrieve all **properties of Paul Auster**


The workload should


1. Identify the BGP for obtaining the books (with publishing date and genre) published by the two authors

2. Did the authors published a book in the same year? What is the longest period without publishing a book for the two authors? 

3. Did the authors produced, acted or directed a film? If so, did they write the screenplay?

4. How many films were derived from the books of these two authors? 

5. Which author won more literature-related awards? Have they ever being nominated for a Nobel award? 

In [1]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT( ?obj)
WHERE { 
wd:Q214642 ?rel ?obj  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '191')]


1

In [9]:
#SBAGLIATA
#here i find all the written work of Paul Aster
queryString = """
SELECT DISTINCT ?book ?pname 
WHERE { 
?book ?r wd:Q47461344.
wd:Q214642 ?rel ?book  . 

?book <http://schema.org/name> ?pname
} 

"""

print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'The Book of Illusions')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('pname', 'Man in the Dark')]
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('pname', 'The New York Trilogy')]
[('book', 'http://www.wikidata.org/entity/Q1821899'), ('pname', 'Leviathan')]


4

In [15]:
#SBAGLIATA
#here i find all the writtenworks of the two Authors

queryString = """
SELECT DISTINCT ?book ?pname
WHERE {
{
?book ?r wd:Q47461344.
wd:Q214642 ?rel ?book  . 

?book <http://schema.org/name> ?pname
} 
UNION
{
?book ?r wd:Q47461344.
wd:Q190379 ?rel ?book  . 

?book <http://schema.org/name> ?pname

}
}
"""

print("Results")
run_query(queryString)




Results
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'The Book of Illusions')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('pname', 'Man in the Dark')]
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('pname', 'The New York Trilogy')]
[('book', 'http://www.wikidata.org/entity/Q1821899'), ('pname', 'Leviathan')]
[('book', 'http://www.wikidata.org/entity/Q306619'), ('pname', 'Atonement')]
[('book', 'http://www.wikidata.org/entity/Q1198186'), ('pname', 'The Cement Garden')]


6

In [18]:

#here i get all the relationship of books
queryString = """
SELECT DISTINCT ?book ?pname ?relationship
WHERE {

?book  ?relationship ?obg.
?book ?r wd:Q47461344.
wd:Q214642 ?rel ?book  . 

?relationship <http://schema.org/name> ?pname

}
"""

print("Results")
run_query(queryString)


Results
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'publisher'), ('relationship', 'http://www.wikidata.org/prop/direct/P123')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'genre'), ('relationship', 'http://www.wikidata.org/prop/direct/P136')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'Encyclopædia Britannica Online ID'), ('relationship', 'http://www.wikidata.org/prop/direct/P1417')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'title'), ('relationship', 'http://www.wikidata.org/prop/direct/P1476')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'follows'), ('relationship', 'http://www.wikidata.org/prop/direct/P155')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'followed by'), ('relationship', 'http://www.wikidata.org/prop/direct/P156')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'OCLC control number'), ('relationship', 'http://www.wikidata.org/prop/direct

87

In [49]:
#in the previous query i got the iri for author P50
#here i get the IRI for book using the data found in the relationship before

queryString = """
SELECT DISTINCT ?book 
WHERE {

wd:Q581485  wdt:P31  ?book.


}
"""

print("Results")
run_query(queryString)


Results
[('book', 'http://www.wikidata.org/entity/Q47461344')]
[('book', 'http://www.wikidata.org/entity/Q571')]


2

In [ ]:
#i get the iri for book:Q571

In [50]:
#here i find the books of the 2 authors

queryString = """
SELECT DISTINCT ?book ?pname
WHERE {
{
?book wdt:P31 wd:Q571.
?book wdt:P50 wd:Q214642.

?book <http://schema.org/name> ?pname
} 
UNION
{

?book wdt:P31 wd:Q571.
?book wdt:P50 wd:Q190379.
 

?book <http://schema.org/name> ?pname

}
}
"""

print("Results")
run_query(queryString)



Results
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'The Book of Illusions')]
[('book', 'http://www.wikidata.org/entity/Q27940587'), ('pname', 'Nutshell')]


2

In [56]:
#here i try to find all the works of the 2 authors so all the works where they are considered the authors (authors is referred only to written works)
#i consider both written works and literary works etc
queryString = """
SELECT DISTINCT ?book ?pname
WHERE {
{

?book wdt:P50 wd:Q214642.


?book <http://schema.org/name> ?pname
} 
UNION
{


?book wdt:P50 wd:Q190379.
 

?book <http://schema.org/name> ?pname

}
}
"""

print("Results")
run_query(queryString)


Results
[('book', 'http://www.wikidata.org/entity/Q1951470'), ('pname', 'Mr. Vertigo')]
[('book', 'http://www.wikidata.org/entity/Q492991'), ('pname', 'Timbuktu')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'The Book of Illusions')]
[('book', 'http://www.wikidata.org/entity/Q526792'), ('pname', 'Travels in the Scriptorium')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('pname', 'Man in the Dark')]
[('book', 'http://www.wikidata.org/entity/Q1322083'), ('pname', 'Sunset Park')]
[('book', 'http://www.wikidata.org/entity/Q1584652'), ('pname', 'Invisible')]
[('book', 'http://www.wikidata.org/entity/Q28846426'), ('pname', '4 3 2 1')]
[('book', 'http://www.wikidata.org/entity/Q60412205'), ('pname', 'Sunset Park')]
[('book', 'http://www.wikidata.org/entity/Q16386002'), ('pname', 'Here and Now: Letters, 2008-2011')]
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('pname', 'The New York Trilogy')]
[('book', 'http://www.wikidata.org/entity/Q1506280'), ('pname'

53

In [57]:
#here i find all the writtenworks of the two Authors with publishing data : PGB P577 and genre  PGB P136 which i found in the query before
#SBAGLIATA NON GUARDARE
queryString = """
SELECT DISTINCT ?book ?pname ?data ?genre
WHERE {
{
?book wdt:P136 ?genre.
?book  wdt:P577 ?data.
?book ?r wd:Q47461344.
wd:Q214642 ?rel ?book  . 

?book <http://schema.org/name> ?pname
} 
UNION
{
?book wdt:P136 ?genre.
?book  wdt:P577 ?data.
?book wdt:P50 wd:Q190379. 

?book <http://schema.org/name> ?pname

}
}
"""

print("Results")
run_query(queryString)



Results
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'The Book of Illusions'), ('data', '2002-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q8261')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('pname', 'Man in the Dark'), ('data', '2008-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q8261')]
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('pname', 'The New York Trilogy'), ('data', '1987-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q8261')]
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('pname', 'The New York Trilogy'), ('data', '1987-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q6585139')]
[('book', 'http://www.wikidata.org/entity/Q1821899'), ('pname', 'Leviathan'), ('data', '1992-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q208505')]
[('book', 'http://www.wikidata.org/entity/Q1821899'), ('pname', 'Leviathan'), ('data', '1992-01-01T00:00:00Z'), ('genre', 'http://www

24

In [12]:
#here i find all the writtenworks of the two Authors with publishing data : PGB P577 and genre  PGB P136 which i found in the query before

queryString = """
SELECT DISTINCT ?book ?pname ?data ?genre
WHERE {
{
?book wdt:P136 ?genre.
?book  wdt:P577 ?data.
?book wdt:P50 wd:Q214642. 

?book <http://schema.org/name> ?pname
} 
UNION
{
?book wdt:P136 ?genre.
?book  wdt:P577 ?data.
?book wdt:P50 wd:Q190379. 

?book <http://schema.org/name> ?pname

}
}
"""

print("Results")
run_query(queryString)



Results
[('book', 'http://www.wikidata.org/entity/Q1951470'), ('pname', 'Mr. Vertigo'), ('data', '1994-04-05T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q8261')]
[('book', 'http://www.wikidata.org/entity/Q492991'), ('pname', 'Timbuktu'), ('data', '1999-05-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q8261')]
[('book', 'http://www.wikidata.org/entity/Q492991'), ('pname', 'Timbuktu'), ('data', '1999-05-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q149537')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'The Book of Illusions'), ('data', '2002-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q8261')]
[('book', 'http://www.wikidata.org/entity/Q526792'), ('pname', 'Travels in the Scriptorium'), ('data', '2006-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.org/entity/Q8261')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('pname', 'Man in the Dark'), ('data', '2008-01-01T00:00:00Z'), ('genre', 'http://www.wikidata.o

49

In [47]:
#sbagliata
#Did the authors published a book in the same year?
queryString = """
SELECT DISTINCT ?book ?book1 ?data ?data1
WHERE {
{
?book  wdt:P577 ?data.
?book wdt:P50 wd:Q214642. 


} 
UNION
{

?book1  wdt:P577 ?data1.
?book1 wdt:P50 wd:Q190379. 



}
FILTER (year(?data) = year(?data1))
}
"""

print("Results")
run_query(queryString)



Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 22007 Error DT001: Function year needs a datetime, date or time as argument 1, not an arg of type DB_NULL (204)\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-267c3eb429-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT DISTINCT ?book ?book1 ?data ?data1\nWHERE {\n{\n?book  wdt:P577 ?data.\n?book wdt:P50 wd:Q214642. \n\n\n} \nUNION\n{\n\n?book1  wdt:P577 ?data1.\n?book1 wdt:P50 wd:Q190379. \n\n\n\n}\nFILTER (year(?data) = year(?data1))\n}\n'


In [49]:
#Sbagliata
queryString = """
SELECT DISTINCT ?book ?book1 ?data ?data1
WHERE {
{
?book  wdt:P577 ?data.
?book wdt:P50 wd:Q214642. 


} 
UNION
{

?book1  wdt:P577 ?data1.
?book1 wdt:P50 wd:Q190379. 



}
FILTER (?data=?data1)
}
"""

print("Results")
run_query(queryString)




Results
Empty


0

In [3]:
#SBAGLIATA NON GUARDARE
#Did the authors published a book in the same year?

queryString = """
SELECT DISTINCT ?book ?pname ?data 
WHERE {
{

?book  wdt:P577 ?data1.
?book ?r wd:Q47461344.
wd:Q214642 ?rel ?book. 

?book <http://schema.org/name> ?pname
} 
UNION
{

?book  wdt:P577 ?data2.
?book ?r wd:Q47461344.
wd:Q190379 ?rel ?book  . 

?book <http://schema.org/name> ?pname

}
}
"""

print("Results")
run_query(queryString)


Results
[('book', 'http://www.wikidata.org/entity/Q581485'), ('pname', 'The Book of Illusions')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('pname', 'Man in the Dark')]
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('pname', 'The New York Trilogy')]
[('book', 'http://www.wikidata.org/entity/Q1821899'), ('pname', 'Leviathan')]
[('book', 'http://www.wikidata.org/entity/Q306619'), ('pname', 'Atonement')]
[('book', 'http://www.wikidata.org/entity/Q1198186'), ('pname', 'The Cement Garden')]


6

In [50]:
#here i return the books and data for the 2 authors 
#Did the authors published a book in the same year? i get the year of the pubblication for each author 
queryString = """
SELECT DISTINCT ?book ?book1 ?data ?data1
WHERE {
{
?book  wdt:P577 ?data.
?book wdt:P50 wd:Q214642. 


} 
UNION
{

?book1  wdt:P577 ?data1.
?book1 wdt:P50 wd:Q190379. 



}

}
"""

print("Results")
run_query(queryString)



Results
[('book', 'http://www.wikidata.org/entity/Q1951470'), ('data', '1994-04-05T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q492991'), ('data', '1999-05-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('data', '2002-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q526792'), ('data', '2006-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('data', '2008-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1322083'), ('data', '2010-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1584652'), ('data', '2009-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q28846426'), ('data', '2017-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q60412205'), ('data', '2010-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q16386002'), ('data', '2013-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('data', '1987-01-01T00:00:00Z')]
[('book', 'http://www.wi

53

In [ ]:
 #here i can easily see that there are several years in common for example in 2010 they both pubblicated Sunset Park and Solar

In [ ]:
#What is the longest period without publishing a book for the two authors?

In [52]:
#here i get the books and the publication data for Paul Auster
queryString = """
SELECT DISTINCT ?book ?data 
WHERE {

?book  wdt:P577 ?data.
?book wdt:P50 wd:Q214642. 





}ORDER BY DESC(?data)
"""

print("Results")
run_query(queryString)



Results
[('book', 'http://www.wikidata.org/entity/Q28846426'), ('data', '2017-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q17027694'), ('data', '2013-11-19T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q16386002'), ('data', '2013-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q16887077'), ('data', '2012-08-21T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q63105572'), ('data', '2012-05-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1322083'), ('data', '2010-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q60412205'), ('data', '2010-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1584652'), ('data', '2009-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('data', '2008-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q42185883'), ('data', '2008-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q526792'), ('data', '2006-01-01T00:00:00Z')]
[('book', 'http://

30

In [ ]:
#here i can see that the biggest period without pubblication for Paul Auster is between 1994 and 1998 

In [53]:
#here i get the books and the publication data for Ian McEwan
queryString = """
SELECT DISTINCT ?book ?data 
WHERE {

?book  wdt:P577 ?data.
?book wdt:P50 wd:Q190379. 





}ORDER BY DESC(?data)
"""

print("Results")
run_query(queryString)



Results
[('book', 'http://www.wikidata.org/entity/Q64031447'), ('data', '2019-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q90699593'), ('data', '2019-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q27940587'), ('data', '2016-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q60977819'), ('data', '2016-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q18162235'), ('data', '2014-09-02T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q972609'), ('data', '2012-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1531861'), ('data', '2010-03-18T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q451491'), ('data', '2007-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q1723053'), ('data', '2005-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q58883639'), ('data', '2005-01-01T00:00:00Z')]
[('book', 'http://www.wikidata.org/entity/Q306619'), ('data', '2001-01-01T00:00:00Z')]
[('book', 'http://www

23

In [ ]:
#here i can see that the biggest period without pubblications is between 1981 and 1987 for  Ian McEwan

In [ ]:
#Did the authors produced, acted or directed a film? If so, did they write the screenplay? P57 P162

In [31]:
#here i find all the relationship of the two authors

queryString = """
SELECT DISTINCT ?rel ?rname ?b
WHERE { 
{
wd:Q214642 ?rel ?b  . 
}
UNION
{
wd:Q190379 ?rel ?b.
}
?rel <http://schema.org/name> ?rname
} 

"""

print("Results")
run_query(queryString)

Results
[('rel', 'http://www.wikidata.org/prop/direct/P1005'), ('rname', 'Portuguese National Library ID'), ('b', '72675')]
[('rel', 'http://www.wikidata.org/prop/direct/P1006'), ('rname', 'Nationale Thesaurus voor Auteurs ID'), ('b', '072139331')]
[('rel', 'http://www.wikidata.org/prop/direct/P101'), ('rname', 'field of work'), ('b', 'http://www.wikidata.org/entity/Q8242')]
[('rel', 'http://www.wikidata.org/prop/direct/P1015'), ('rname', 'NORAF ID'), ('b', '90118966')]
[('rel', 'http://www.wikidata.org/prop/direct/P106'), ('rname', 'occupation'), ('b', 'http://www.wikidata.org/entity/Q11774202')]
[('rel', 'http://www.wikidata.org/prop/direct/P106'), ('rname', 'occupation'), ('b', 'http://www.wikidata.org/entity/Q14467526')]
[('rel', 'http://www.wikidata.org/prop/direct/P106'), ('rname', 'occupation'), ('b', 'http://www.wikidata.org/entity/Q1622272')]
[('rel', 'http://www.wikidata.org/prop/direct/P106'), ('rname', 'occupation'), ('b', 'http://www.wikidata.org/entity/Q18814623')]
[('rel

321

In [34]:
#from the previous query i get the IRI for occupation P106 so i can look if they where screen writers
#i get all the occupation for the 2 authors


queryString = """
SELECT DISTINCT ?occupation ?oname
WHERE { 
{
wd:Q214642 wdt:P106 ?occupation. 
}
UNION
{
wd:Q190379 wdt:P106 ?occupation.
}
?occupation <http://schema.org/name> ?oname
} 

"""

print("Results")
run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q11774202'), ('oname', 'essayist')]
[('occupation', 'http://www.wikidata.org/entity/Q14467526'), ('oname', 'linguist')]
[('occupation', 'http://www.wikidata.org/entity/Q1622272'), ('oname', 'university teacher')]
[('occupation', 'http://www.wikidata.org/entity/Q18814623'), ('oname', 'autobiographer')]
[('occupation', 'http://www.wikidata.org/entity/Q2526255'), ('oname', 'film director')]
[('occupation', 'http://www.wikidata.org/entity/Q28389'), ('oname', 'screenwriter')]
[('occupation', 'http://www.wikidata.org/entity/Q333634'), ('oname', 'translator')]
[('occupation', 'http://www.wikidata.org/entity/Q36180'), ('oname', 'writer')]
[('occupation', 'http://www.wikidata.org/entity/Q482980'), ('oname', 'author')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('oname', 'poet')]
[('occupation', 'http://www.wikidata.org/entity/Q6625963'), ('oname', 'novelist')]
[('occupation', 'http://www.wikidata.org/entity/Q214917'), ('onam

13

In [ ]:
#i get screenwriter and film producer so they produced a film i get the iri for screen writer Q28389 

In [40]:
#here i get all the relationship with screenwriter Q28389
queryString = """
SELECT DISTINCT ?relation ?rname ?b ?bname
WHERE { 
wd:Q28389 ?relation ?b.
?relation <http://schema.org/name> ?rname.
?b <http://schema.org/name> ?bname.
} 

"""

print("Results")
run_query(queryString)

Results
[('relation', 'http://www.wikidata.org/prop/direct/P31'), ('rname', 'instance of'), ('b', 'http://www.wikidata.org/entity/Q4220920'), ('bname', 'filmmaking occupation')]
[('relation', 'http://www.wikidata.org/prop/direct/P910'), ('rname', "topic's main category"), ('b', 'http://www.wikidata.org/entity/Q3920019'), ('bname', 'Category:Screenwriters')]
[('relation', 'http://www.wikidata.org/prop/direct/P2868'), ('rname', 'subject has role'), ('b', 'http://www.wikidata.org/entity/Q36180'), ('bname', 'writer')]
[('relation', 'http://www.wikidata.org/prop/direct/P279'), ('rname', 'subclass of'), ('b', 'http://www.wikidata.org/entity/Q36180'), ('bname', 'writer')]
[('relation', 'http://www.wikidata.org/prop/direct/P425'), ('rname', 'field of this occupation'), ('b', 'http://www.wikidata.org/entity/Q3429919'), ('bname', 'screenwriting')]
[('relation', 'http://www.wikidata.org/prop/direct/P1687'), ('rname', 'Wikidata property'), ('b', 'http://www.wikidata.org/entity/P58'), ('bname', 'sc

6

In [ ]:
# i get filmaking occupation Q4220920

In [41]:
#i get all the relationship with filmaking occupation
queryString = """
SELECT DISTINCT ?relation ?rname ?b ?bname
WHERE { 
wd:Q4220920 ?relation ?b.
?relation <http://schema.org/name> ?rname.
?b <http://schema.org/name> ?bname.
} 

"""

print("Results")
run_query(queryString)

Results
[('relation', 'http://www.wikidata.org/prop/direct/P1687'), ('rname', 'Wikidata property'), ('b', 'http://www.wikidata.org/entity/P3092'), ('bname', 'film crew member')]
[('relation', 'http://www.wikidata.org/prop/direct/P3095'), ('rname', 'practiced by'), ('b', 'http://www.wikidata.org/entity/Q1414443'), ('bname', 'filmmaker')]
[('relation', 'http://www.wikidata.org/prop/direct/P279'), ('rname', 'subclass of'), ('b', 'http://www.wikidata.org/entity/Q28640'), ('bname', 'profession')]
[('relation', 'http://www.wikidata.org/prop/direct/P31'), ('rname', 'instance of'), ('b', 'http://www.wikidata.org/entity/Q19361238'), ('bname', 'Wikidata metaclass')]
[('relation', 'http://www.wikidata.org/prop/direct/P1424'), ('rname', "topic's main template"), ('b', 'http://www.wikidata.org/entity/Q5871779'), ('bname', 'Template:Infobox filmmaker')]
[('relation', 'http://www.wikidata.org/prop/direct/P3095'), ('rname', 'practiced by'), ('b', 'http://www.wikidata.org/entity/Q3406858'), ('bname', '

7

In [42]:
#i get all the relationship with filmaker that i found in the previous query Q1414443
queryString = """
SELECT DISTINCT ?relation ?rname ?b ?bname
WHERE { 
wd:Q1414443 ?relation ?b.
?relation <http://schema.org/name> ?rname.
?b <http://schema.org/name> ?bname.
} 

"""

print("Results")
run_query(queryString)

Results
[('relation', 'http://www.wikidata.org/prop/direct/P31'), ('rname', 'instance of'), ('b', 'http://www.wikidata.org/entity/Q4220920'), ('bname', 'filmmaking occupation')]
[('relation', 'http://www.wikidata.org/prop/direct/P31'), ('rname', 'instance of'), ('b', 'http://www.wikidata.org/entity/Q47728'), ('bname', 'hobby')]
[('relation', 'http://www.wikidata.org/prop/direct/P279'), ('rname', 'subclass of'), ('b', 'http://www.wikidata.org/entity/Q2526255'), ('bname', 'film director')]
[('relation', 'http://www.wikidata.org/prop/direct/P279'), ('rname', 'subclass of'), ('b', 'http://www.wikidata.org/entity/Q483501'), ('bname', 'artist')]
[('relation', 'http://www.wikidata.org/prop/direct/P361'), ('rname', 'part of'), ('b', 'http://www.wikidata.org/entity/Q932586'), ('bname', 'filmmaking')]
[('relation', 'http://www.wikidata.org/prop/direct/P279'), ('rname', 'subclass of'), ('b', 'http://www.wikidata.org/entity/Q3391743'), ('bname', 'visual artist')]
[('relation', 'http://www.wikidata

10

In [43]:
#i get all the relationship whit filmaking Q932586
queryString = """
SELECT DISTINCT ?relation ?rname ?b ?bname
WHERE { 
wd:Q932586 ?relation ?b.
?relation <http://schema.org/name> ?rname.
?b <http://schema.org/name> ?bname.
} 

"""

print("Results")
run_query(queryString)

Results
[('relation', 'http://www.wikidata.org/prop/direct/P279'), ('rname', 'subclass of'), ('b', 'http://www.wikidata.org/entity/Q21198548'), ('bname', 'audiovisual production')]
[('relation', 'http://www.wikidata.org/prop/direct/P1056'), ('rname', 'product or material produced'), ('b', 'http://www.wikidata.org/entity/Q11424'), ('bname', 'film')]
[('relation', 'http://www.wikidata.org/prop/direct/P31'), ('rname', 'instance of'), ('b', 'http://www.wikidata.org/entity/Q11862829'), ('bname', 'academic discipline')]
[('relation', 'http://www.wikidata.org/prop/direct/P1343'), ('rname', 'described by source'), ('b', 'http://www.wikidata.org/entity/Q2657718'), ('bname', 'Armenian Soviet Encyclopedia')]
[('relation', 'http://www.wikidata.org/prop/direct/P3095'), ('rname', 'practiced by'), ('b', 'http://www.wikidata.org/entity/Q3282637'), ('bname', 'film producer')]
[('relation', 'http://www.wikidata.org/prop/direct/P279'), ('rname', 'subclass of'), ('b', 'http://www.wikidata.org/entity/Q5908

14

In [ ]:
#so i get the IRI for film Q11424

In [36]:
#using the iri film i get the films and their profession in those films
queryString = """
SELECT DISTINCT ?job ?film ?jname
WHERE { 
{
?film wdt:P31 wd:Q11424. 
?film ?job  wd:Q214642.
}
UNION
{
?film wdt:P31 wd:Q11424. 
?film  ?job  wd:Q190379.

}

?job <http://schema.org/name> ?jname
} 

"""

print("Results")
run_query(queryString)

Results
[('job', 'http://www.wikidata.org/prop/direct/P161'), ('film', 'http://www.wikidata.org/entity/Q2915460'), ('jname', 'cast member')]
[('job', 'http://www.wikidata.org/prop/direct/P161'), ('film', 'http://www.wikidata.org/entity/Q2364684'), ('jname', 'cast member')]
[('job', 'http://www.wikidata.org/prop/direct/P162'), ('film', 'http://www.wikidata.org/entity/Q3213673'), ('jname', 'producer')]
[('job', 'http://www.wikidata.org/prop/direct/P170'), ('film', 'http://www.wikidata.org/entity/Q3213673'), ('jname', 'creator')]
[('job', 'http://www.wikidata.org/prop/direct/P57'), ('film', 'http://www.wikidata.org/entity/Q1877215'), ('jname', 'director')]
[('job', 'http://www.wikidata.org/prop/direct/P57'), ('film', 'http://www.wikidata.org/entity/Q2478977'), ('jname', 'director')]
[('job', 'http://www.wikidata.org/prop/direct/P57'), ('film', 'http://www.wikidata.org/entity/Q653447'), ('jname', 'director')]
[('job', 'http://www.wikidata.org/prop/direct/P57'), ('film', 'http://www.wikidat

28

In [46]:

    #i get the iri producer P162 director P57 cast member P161 and screenwriter P58
#Here i get the films directed, acted or produced by Paul austen where he was also the screen writer


queryString = """
SELECT DISTINCT ?film ?jname
WHERE { 
{
?film wdt:P31 wd:Q11424. 
?film  wdt:P162 wd:Q214642.
}
UNION
{
?film wdt:P31 wd:Q11424. 
?film  wdt:P57 wd:Q214642.
}
UNION
{
?film wdt:P31 wd:Q11424. 
?film  wdt:P161 wd:Q214642.
}
?film <http://schema.org/name> ?jname
FILTER EXISTS{?film wdt:P58 wd:Q214642}.
} 

"""

print("Results")
run_query(queryString)
    

Results
[('film', 'http://www.wikidata.org/entity/Q3213673'), ('jname', 'The Inner Life of Martin Frost')]
[('film', 'http://www.wikidata.org/entity/Q2478977'), ('jname', 'Blue in the Face')]
[('film', 'http://www.wikidata.org/entity/Q653447'), ('jname', 'Smoke')]
[('film', 'http://www.wikidata.org/entity/Q1877215'), ('jname', 'Lulu on the Bridge')]
[('film', 'http://www.wikidata.org/entity/Q2915460'), ('jname', 'Act of God')]
[('film', 'http://www.wikidata.org/entity/Q2364684'), ('jname', 'The Music of Chance')]


6

In [47]:
#Here i get the films directed, acted or produced by Ian McEwan where he was also the screen writer


queryString = """
SELECT DISTINCT ?film ?jname
WHERE { 
{
?film wdt:P31 wd:Q11424. 
?film  wdt:P162 wd:Q190379.
}
UNION
{
?film wdt:P31 wd:Q11424. 
?film  wdt:P57 wd:Q190379.
}
UNION
{
?film wdt:P31 wd:Q11424. 
?film  wdt:P161 wd:Q190379.
}
?film <http://schema.org/name> ?jname
FILTER EXISTS{?film wdt:P58 wd:Q190379}.
} 

"""

print("Results")
run_query(queryString)
    

Results
[('film', 'http://www.wikidata.org/entity/Q1626186'), ('jname', 'Atonement')]


1

In [59]:
#here i get the relationship whit the film  cement garden Q1916423 found before


queryString = """
SELECT DISTINCT ?rel ?jname
WHERE { 


wd:Q1916423 ?rel ?b.

?rel <http://schema.org/name> ?jname.
} 

"""

print("Results")
run_query(queryString)




Results
[('rel', 'http://www.wikidata.org/prop/direct/P1237'), ('jname', 'Box Office Mojo film ID (former scheme)')]
[('rel', 'http://www.wikidata.org/prop/direct/P1258'), ('jname', 'Rotten Tomatoes ID')]
[('rel', 'http://www.wikidata.org/prop/direct/P1265'), ('jname', 'AlloCiné film ID')]
[('rel', 'http://www.wikidata.org/prop/direct/P136'), ('jname', 'genre')]
[('rel', 'http://www.wikidata.org/prop/direct/P144'), ('jname', 'based on')]
[('rel', 'http://www.wikidata.org/prop/direct/P1476'), ('jname', 'title')]
[('rel', 'http://www.wikidata.org/prop/direct/P1562'), ('jname', 'AllMovie title ID')]
[('rel', 'http://www.wikidata.org/prop/direct/P161'), ('jname', 'cast member')]
[('rel', 'http://www.wikidata.org/prop/direct/P1874'), ('jname', 'Netflix ID')]
[('rel', 'http://www.wikidata.org/prop/direct/P1970'), ('jname', 'MovieMeter film ID')]
[('rel', 'http://www.wikidata.org/prop/direct/P2047'), ('jname', 'duration')]
[('rel', 'http://www.wikidata.org/prop/direct/P2346'), ('jname', 'Elon

53

In [ ]:
#i found the IRI for based on which i will use now P144

In [75]:
#How many films were derived from the books of these two authors?



queryString = """
SELECT DISTINCT  COUNT(?film) as ?NumFilms 
WHERE { 
{
?film wdt:P31 wd:Q11424. 
?film wdt:P144 ?book.
?book wdt:P50 wd:Q214642.
}
UNION
{
?film wdt:P31 wd:Q11424. 
?film wdt:P144 ?book.
?book wdt:P50 wd:Q190379.

}


} 

"""

print("Results")
run_query(queryString)

Results
[('NumFilms', '4')]


1

In [ ]:
#Which author won more literature-related awards? Have they ever being nominated for a Nobel award?

In [76]:
#here i find all the relationship whit the award described by the iri Q3405076 found before 


queryString = """
SELECT DISTINCT  ?rel ?obj ?jname ?oname
WHERE { 

wd:Q3405076 ?rel ?obj.
?rel <http://schema.org/name> ?jname.
?obj <http://schema.org/name> ?oname.
} 

"""

print("Results")
run_query(queryString)


Results
[('rel', 'http://www.wikidata.org/prop/direct/P361'), ('obj', 'http://www.wikidata.org/entity/Q58352'), ('jname', 'part of'), ('oname', 'Prix Médicis')]
[('rel', 'http://www.wikidata.org/prop/direct/P17'), ('obj', 'http://www.wikidata.org/entity/Q142'), ('jname', 'country'), ('oname', 'France')]
[('rel', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q378427'), ('jname', 'subclass of'), ('oname', 'literary award')]
[('rel', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q38033430'), ('jname', 'instance of'), ('oname', 'class of award')]
[('rel', 'http://www.wikidata.org/prop/direct/P112'), ('obj', 'http://www.wikidata.org/entity/Q3169349'), ('jname', 'founded by'), ('oname', 'Jean-Pierre Giraudoux')]
[('rel', 'http://www.wikidata.org/prop/direct/P112'), ('obj', 'http://www.wikidata.org/entity/Q15139267'), ('jname', 'founded by'), ('oname', 'Gala Barbisan')]


6

In [ ]:
#in the previous query i found the iri for literary awards Q378427 and the iri for subclass of P279, i found before the iri for award received P166

In [86]:
#here i found the literary awards received by the 2 authors

queryString = """
SELECT DISTINCT ?award ?pname
WHERE {
{
?award wdt:P279 wd:Q378427.
wd:Q214642 wdt:P166 ?award.

?award <http://schema.org/name> ?pname.
} 
UNION
{
?award wdt:P279 wd:Q378427.
wd:Q190379 wdt:P166 ?award.
 

?award <http://schema.org/name> ?pname

}
UNION
{
?award wdt:P31 wd:Q378427.
wd:Q190379 wdt:P166 ?award.
 

?award <http://schema.org/name> ?pname

}
UNION
{
?award wdt:P31 wd:Q378427.
wd:Q214642 wdt:P166 ?award.

?award <http://schema.org/name> ?pname.
}
}
"""

print("Results")
run_query(queryString)



Results
[('award', 'http://www.wikidata.org/entity/Q3405076'), ('pname', 'Prix Médicis for foreign literature')]
[('award', 'http://www.wikidata.org/entity/Q39477935'), ('pname', 'Fellow of the Royal Society of Literature')]
[('award', 'http://www.wikidata.org/entity/Q160082'), ('pname', 'Booker Prize')]
[('award', 'http://www.wikidata.org/entity/Q392586'), ('pname', 'James Tait Black Memorial Prize')]
[('award', 'http://www.wikidata.org/entity/Q5709276'), ('pname', 'Helmerich Award')]
[('award', 'http://www.wikidata.org/entity/Q3404580'), ('pname', 'Prix Femina étranger')]
[('award', 'http://www.wikidata.org/entity/Q4936657'), ('pname', 'Bodley Medal')]
[('award', 'http://www.wikidata.org/entity/Q7462809'), ('pname', 'Shakespeare Prize')]
[('award', 'http://www.wikidata.org/entity/Q3332454'), ('pname', 'Princess of Asturias Literary Prize')]
[('award', 'http://www.wikidata.org/entity/Q1188661'), ('pname', 'PEN/Faulkner Award for Fiction')]


10

In [87]:
#here i found the literature related awards for wd:Q214642 Paul Auster
queryString = """
SELECT DISTINCT ?award ?pname
WHERE {
{
?award wdt:P279 wd:Q378427.
wd:Q214642 wdt:P166 ?award.

?award <http://schema.org/name> ?pname.
} 

UNION
{
?award wdt:P31 wd:Q378427.
wd:Q214642 wdt:P166 ?award.

?award <http://schema.org/name> ?pname.
}
}
"""

print("Results")
run_query(queryString)



Results
[('award', 'http://www.wikidata.org/entity/Q3405076'), ('pname', 'Prix Médicis for foreign literature')]
[('award', 'http://www.wikidata.org/entity/Q3332454'), ('pname', 'Princess of Asturias Literary Prize')]
[('award', 'http://www.wikidata.org/entity/Q1188661'), ('pname', 'PEN/Faulkner Award for Fiction')]


3

In [89]:
#here i found the literary awards for wd:Q190379 Ian McEwan

queryString = """
SELECT DISTINCT ?award ?pname
WHERE {


{
?award wdt:P279 wd:Q378427.
wd:Q190379 wdt:P166 ?award.
 

?award <http://schema.org/name> ?pname

}
UNION
{
?award wdt:P31 wd:Q378427.
wd:Q190379 wdt:P166 ?award.
 

?award <http://schema.org/name> ?pname

}

}
"""

print("Results")
run_query(queryString)


Results
[('award', 'http://www.wikidata.org/entity/Q39477935'), ('pname', 'Fellow of the Royal Society of Literature')]
[('award', 'http://www.wikidata.org/entity/Q160082'), ('pname', 'Booker Prize')]
[('award', 'http://www.wikidata.org/entity/Q392586'), ('pname', 'James Tait Black Memorial Prize')]
[('award', 'http://www.wikidata.org/entity/Q5709276'), ('pname', 'Helmerich Award')]
[('award', 'http://www.wikidata.org/entity/Q3404580'), ('pname', 'Prix Femina étranger')]
[('award', 'http://www.wikidata.org/entity/Q4936657'), ('pname', 'Bodley Medal')]
[('award', 'http://www.wikidata.org/entity/Q7462809'), ('pname', 'Shakespeare Prize')]


7

In [ ]:
#so Ian McEwan won more literature related awards

In [ ]:
# Have they ever being nominated for a Nobel award? before i get the iri for nominated for P1411 now i try to get all the things that theywhere nominated for

In [90]:
queryString = """
SELECT DISTINCT  ?prize ?pname
WHERE { 
{
wd:Q214642 wdt:P1411 ?prize.
}
UNION
{
wd:Q190379 wdt:P1411 ?prize.

}
?prize <http://schema.org/name> ?pname

} 

"""

print("Results")
run_query(queryString)

Results
[('prize', 'http://www.wikidata.org/entity/Q628637'), ('pname', "European Film Award – People's Choice Award for Best European Film")]
[('prize', 'http://www.wikidata.org/entity/Q160082'), ('pname', 'Booker Prize')]
[('prize', 'http://www.wikidata.org/entity/Q1362723'), ('pname', 'International IMPAC Dublin Literary Award')]
[('prize', 'http://www.wikidata.org/entity/Q2052291'), ('pname', 'International Booker Prize')]


4

In [ ]:
#no they have never been nominated for a literature award